## Deneme

In [5]:
a = None
print(a), a + 2

None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import DatasetDict, Dataset
import re

In [ ]:
prefix = "data/tr_wiki67/trwiki-67."
file_paths = [ f"{prefix}{postfix}.raw" for postfix in ["test", "train", "val"] ]
file_paths

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download("punkt")

In [ ]:
example_sentence = "Türkiye-İran55 ilişkileri.55_55 - 987@"
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(example_sentence)
words

## Test for prepare.py script (get_raw_dataset functionality)

In [ ]:
import data.tr_wiki67.prepare as prepare

prepare.get_raw_dataset()

In [ ]:
from datasets import load_from_disk

ds = load_from_disk("data/tr_wiki67/raw_dataset")

In [ ]:
print(ds["titles"][21])
print(ds["examples"][21])
print(ds["length"][21])

In [ ]:
print(ds["titles"][21])
print(ds["examples"][21])
print(ds["length"][21])

## Build Train and Save Tokenizer

In [28]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFKC(),
     normalizers.Lowercase()]
)

# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.WhitespaceSplit(), 
                                                   pre_tokenizers.Digits(individual_digits=True),
                                                   pre_tokenizers.Punctuation()])


special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

trainer = trainers.WordPieceTrainer(vocab_size=25_000, special_tokens=special_tokens, min_frequency=5,
                                    continuing_subword_prefix="##", limit_alphabet=500)





In [29]:
print(tokenizer.normalizer.normalize_str("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?"))


?   merhabalar    555555 benim8 adım! ! .muhammet can gümüşsuüğği̇i̇çç 55  ة الزلزلة ?? ?


In [30]:
tokenizer.pre_tokenizer.pre_tokenize_str("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?")

[('?', (0, 1)),
 ('Merhabalar', (4, 14)),
 ('5', (18, 19)),
 ('5', (19, 20)),
 ('5', (20, 21)),
 ('5', (21, 22)),
 ('5', (22, 23)),
 ('5', (23, 24)),
 ('benim', (25, 30)),
 ('8', (30, 31)),
 ('adım', (32, 36)),
 ('!', (36, 37)),
 ('!', (38, 39)),
 ('.', (40, 41)),
 ('Muhammet', (41, 49)),
 ('Can', (50, 53)),
 ('GümüşsuÜğĞİİÇç', (54, 68)),
 ('5', (69, 70)),
 ('5', (70, 71)),
 ('ة', (73, 74)),
 ('الزلزلة', (75, 82)),
 ('?', (83, 84)),
 ('?', (84, 85)),
 ('?', (86, 87))]

In [31]:
from data.tr_wiki67.prepare import get_raw_dataset

dataset = get_raw_dataset()

def get_example_batch():
    """
    generator func
    will be used when training tokenizer
    """
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["examples"]

In [ ]:
tokenizer.train_from_iterator(get_example_batch(), trainer=trainer)

In [32]:
encoding = tokenizer.encode("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?")
print(encoding.tokens)

Exception: WordPiece error: Missing [UNK] token from the vocabulary

In [ ]:
print(encoding)
print(encoding.ids)
print(encoding.type_ids)
print(encoding.tokens)
print(encoding.offsets)
print(encoding.attention_mask)
print(encoding.special_tokens_mask)
print(encoding.overflowing)

In [33]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]") 

print(cls_token_id, sep_token_id)


tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)]
)

tokenizer.decoder = decoders.WordPiece(prefix="##")

None None


TypeError: argument 'special_tokens': Expected Union[Tuple[str, int], Tuple[int, str], dict]

In [ ]:
encoding = tokenizer.encode(" merhaba benim adım name..." , " AS gardaşım peki benim adım ne...")
print(encoding)
print(encoding.ids)
print(encoding.type_ids)
print(encoding.tokens)
print(encoding.offsets)
print(encoding.attention_mask)
print(encoding.special_tokens_mask)
print(encoding.overflowing)

In [36]:
tokenizer = Tokenizer.from_file("tokenizer_wordpiece/tokenizer.json")

tokenizers.Tokenizer

In [ ]:
tokenizer.decode(encoding.ids)

In [ ]:
with open("data/tr_wiki67/trwiki-67.test.raw", "+r") as f:
    content = f.read()
    total_num_char = len(content)
    acc = len(tokenizer.encode(content).tokens)
    print(total_num_char / acc)

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer/tokenizer_55.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

In [ ]:
wrapped_tokenizer.is_fast

In [ ]:
wrapped_tokenizer.save_pretrained("fast_wordpiece")

In [ ]:
test = """4 Haziran 1980'de İzmir'de doğdu. Melih Özakat İlkokulu'na, Özel Çakabey Lisesi'nde ortaokula ve konservatuvarda liseye gitti. 7 yaşında okul hayatına başlamasıyla TRT ve İzmir Devlet Konservatuvarı'nın çocuk korolarında müzik eğitimine başlaması aynı zamana denk gelir. İlk geri vokal deneyimini 12 yaşında Sezen Aksu'yla beraber 1992 yılında gerçekleştirdi. Sezen Aksu'nun yanında solo ve vokal çalışmaları devam ederken okul hayatı İzmir'de devam etmekteydi.
İlk TV programına yine Sezen Aksu'yla 1992 yılında TRT’de çıktı. Geri vokal çalışmalarının yanı sıra bu programda solo bir şarkı da seslendirmesi kamuoyunda büyük ilgi gördü. Büyük yetenek olarak lanse edilen Tuğba Özerk'in sahne ve müzik hayatı tam anlamıyla böyle başladı denilebilir. Okul hayatı bittikten sonra İstanbul’a taşınan sanatçı aralarında Ege, Deniz Seki gibi isimler bulunan sanatçılara geri vokal yaptı. Daha sonra solo sahne çalışmalarına başlayan Özerk, 2002 yılının Aralık ayında "Dün Gibi" adlı ilk solo albümünü çıkardı. 'Aşk yarası' adlı şarkısıyla uzun süre radyolarda liste başlarında yer aldı. Albümle beraber oyunculuk yönünü gösterebilme fırsatı yakalayan sanatçı, atv'nin uzun süredir yayın yapmış olduğu dizisi "Böyle mi olacaktı"da bir sene kadar rol aldı. İlk albümünde bir tane bestesi olan Özerk üç senelik boşlukta birçok besteye de imza attı.6 Temmuz 2005 tarihinde 'Lo Lo Lo' adlı ikinci solo albümünü çıkardı.'Lo Lo Lo' adlı şarkısıyla uzun süre radyolarda liste başı oldu. İkinci albümünde bir tane bestesi olan iki senelik boşlukta birçok besteye de imza attı
"""
encoding = wrapped_tokenizer(test)
print(len(encoding.tokens()))   # our tokenizer does not have any limit-boundry about block_size
encoding.tokens()[:254]

## Prepare Data For (NSP-MLM) Task

In [1]:
from datasets import Dataset, load_from_disk, load_dataset
from nltk.tokenize import sent_tokenize
import os

In [ ]:
# nsp_mlm_dataset = { "a_b_text": [], "x": [], "y": [], "att_mask": [], "token_type": [], "isNext": [] }

isNext = None
x = []
y = []
attention_mask = []
token_type = []
token_id = []
a_b_text = []

block_size = 255
A_segment_flag = True

In [ ]:
raw_table = load_from_disk("data/tr_wiki67/raw_dataset")

In [407]:
def create_pairs(raw_row):
    # split example by sentences
    list_of_sentences = sent_tokenize(raw_row["examples"])
    A = []
    B = []

    for pair_index in range(0, len(list_of_sentences)-1):
        A.append(list_of_sentences[pair_index])
        B.append(list_of_sentences[pair_index + 1])

    return {"A": A, "B": B}


def create_a_b_table(raw_table_path):
    """
    TODO: ara kayıt işlemleri yap (save as arrow), hali hazırda var ise yükle ve döndür, examples, titles, length kolonlarını sil
    """
    # load raw dataset from disk
    raw_table = load_from_disk(raw_table_path)
    # # bu satır geçici! test'i hızlı yapabilmek için
    # raw_table = Dataset.from_dict(raw_table[:5_000])
    # every example needs at least 2 sentence
    raw_table_filtered = raw_table.filter(lambda row:  len(sent_tokenize(row["examples"], language="turkish")) > 1)


    # create table that contains A, B kols (problem: row of this table has list of sentences not just one sentence for each pair)
    a_b_list_table = raw_table_filtered.map(create_pairs)


    # 
    A, B = [], []
    for A_list, B_list in zip(a_b_list_table["A"], a_b_list_table["B"]):
        A += A_list
        B += B_list
    
    a_b_table = {"A":A, "B":B}
    a_b_table = Dataset.from_dict(a_b_table)

    return a_b_table


In [ ]:
raw_table_path = "data/tr_wiki67/raw_dataset"
ab_table_path = "deneme_ab_tablosu"


# if table is not already exists
if os.path.exists(ab_table_path) == False:
    print("table is not already exists, will be created and saved to disk...")
    
    # create table
    a_b_table = create_a_b_table(raw_table_path)

    # save created table
    a_b_table.save_to_disk(ab_table_path)

# ab table is already exists
else:
    print("table is already exists, will be loaded from disk...")
    # load table from disk
    a_b_table = load_from_disk(ab_table_path)



In [ ]:
a_b_table

In [ ]:
temp = a_b_table[:1000]

In [ ]:
for i in range(5):
    print(f"A: {temp['A'][i]}, \nB: {temp['B'][i]}\n\n")

## NSP-MLM PREP

In [1]:
from datasets import load_from_disk, Dataset
import os
import random

from torch.utils.data import DataLoader
import torch

import re

from tokenizers import (
    pre_tokenizers,
    processors,
    Tokenizer,
)

In [2]:
# GLOBAL VARS  
trwiki_path_prefix = "data/tr_wiki67/"
raw_table_path = trwiki_path_prefix + "raw_dataset"
ab_table_path = trwiki_path_prefix + "deneme_ab_tablosu"
nsp_mlm_table_path = trwiki_path_prefix + "nsp_mlm_table"
block_size = 255

tokenizer = Tokenizer.from_file("tokenizer_wordpiece/tokenizer.json")

cls_id = tokenizer.token_to_id("[CLS]")
sep_id = tokenizer.token_to_id("[SEP]")
pad_id = tokenizer.token_to_id("[PAD]")
mask_id = tokenizer.token_to_id("[MASK]")

# if table is not already exists
if os.path.exists(ab_table_path) == False:
    print("table is not already exists, will be created and saved to disk...")
    
    # create table
    a_b_table = create_a_b_table(raw_table_path)

    # save created table
    a_b_table.save_to_disk(ab_table_path)

# ab table is already exists
else:
    print("table is already exists, will be loaded from disk...")
    # load table from disk
    a_b_table = load_from_disk(ab_table_path)

# this chunk will be used for finding random word or sentence
sentences_list = a_b_table[:10_000]
sentences_list = sentences_list["A"] + sentences_list["B"]  # list + list -> list

table is already exists, will be loaded from disk...


In [3]:
def get_random_sentence():
    return sentences_list[random.randint(0, len(sentences_list) - 1)]

def get_random_word_tokens(num_tokens):
    limit = 100
    trial = 0
    pattern = r'\d'

    while trial < limit:
        trial += 1
        words = get_random_sentence().split(" ")
        for word in words:
            # if word is number like 1881, 2005, ...
            # jumpt to next word
            if re.search(pattern, word):
                # print("number!! ", word)
                continue

            cand_word_token_ids = tokenizer.encode(word, add_special_tokens=False).ids
            if len(cand_word_token_ids) == num_tokens:
                # print(word)
                return cand_word_token_ids
            
    # trial limit exceeded return None
    return None

In [4]:
t = get_random_word_tokens(1)
if  t is None:
    print(None)
else:
    print(t), print(tokenizer.decode(t))

[11970]
tanrıça


In [5]:
get_random_sentence()

'İsviçre, Avrupa Birliği üyesi olmamasına rağmen, "ne bir siyasi birimin bir devlet olabilmesi için tanınması gerekmiyor, ne de bir devletin diğerini tanıma yükümlülüğü bulunmadığını belirterek belirtilen bu ilkeye bağlı kalıyor.'

In [6]:
a_b_table

Dataset({
    features: ['A', 'B'],
    num_rows: 4119470
})

In [19]:
gecici_dataset = {"a":[5,3,66,6510,4,None,45,444,None]}
gecici_dataset

{'a': [5, 3, 66, 6510, 4, None, 45, 444, None]}

In [21]:
gecici_dataset = Dataset.from_dict(gecici_dataset)

AttributeError: 'Dataset' object has no attribute 'items'

In [24]:
gecici_dataset

Dataset({
    features: ['a'],
    num_rows: 9
})

In [25]:
gecici_dataset.filter(lambda x: x["a"] != None)

Filter:   0%|          | 0/9 [00:00<?, ? examples/s]

Dataset({
    features: ['a'],
    num_rows: 7
})

In [26]:
gecici_dataset

Dataset({
    features: ['a'],
    num_rows: 9
})

In [32]:
def del_none_filter(row):
    return row["X"] != None


def create_nsp_mlm_table(ab_row):
    
    isNext = random.random() > 0.5
    x = []  # input sequence
    y = []  # target sequence
    
    if isNext == False:
        rand_b = get_random_sentence()
        ab_row["B"] = rand_b

    encoding = tokenizer.encode(ab_row["A"], ab_row["B"])
    ab_len = len(encoding.tokens) 


    if ab_len > block_size:
        return {"X": None, "Y": None, "Attention_mask": None, "Token_type_id":None, "isNext": None}
    
    seq = tokenizer.decode(encoding.ids, skip_special_tokens=False).split(" ")

    for word in seq:

        if word == "[CLS]":
            x.append(cls_id)
            y.append(int(isNext))

        elif word == "[SEP]":
            x.append(sep_id)
            y.append(pad_id)
        
        else:
            tokens = tokenizer.encode(word, add_special_tokens=False).ids
            
            prob = random.random()
            if prob < 0.15 :


                prob_inner = random.random()
                if prob_inner < 0.8:
                    # mask condition
                    x += [mask_id for token in tokens]
                    y += tokens
                    # print("mask condition: ")
                    # print("x: ", tokenizer.decode([mask_id for token in tokens], skip_special_tokens=False))
                    # print("y: ", tokenizer.decode(tokens, skip_special_tokens=False))
                               
                elif prob_inner < 0.9:
                    # corrupt condition
                    random_word_tokens = get_random_word_tokens(num_tokens=len(tokens))

                    if random_word_tokens is None:
                        # could not found word that has the same token length as corrupted word
                        print("could not found word")
                        print("len of tokes: ", len(tokens))
                        return {"X": None, "Y": None, "Attention_mask": None, "Token_type_id":None, "isNext": None}
                    
                    x += random_word_tokens
                    y += tokens
                    # print("corrupt condition:")
                    # print("x: ", tokenizer.decode(random_word_tokens, skip_special_tokens=False))
                    # print("y: ", tokenizer.decode(tokens, skip_special_tokens=False))
                
                else:
                    # identity condition
                    x += tokens
                    y += tokens
                    # print("identity condition:")
                    # print("x: ", tokenizer.decode(tokens, skip_special_tokens=False))
                    # print("y: ", tokenizer.decode(tokens, skip_special_tokens=False))
                    
                
            else:
                x += tokens 
                y += [pad_id for token in tokens]
            

    assert len(x) == len(y), "[ERROR] there is some mistake, x and y should be the same length!"

    num_pad = block_size - len(x)
    x += [pad_id for each in range(num_pad)]
    y += [pad_id for each in range(num_pad)]
    
    first_sep_idx = x.index(sep_id)
    token_type_id = [ 0 if i <= first_sep_idx else 1 for i in range(len(x)) ]
    attention_mask = [ 0 if token_id == pad_id else 1 for token_id in x ]

    # print("\nX: ", tokenizer.decode(x, skip_special_tokens=False),
    #        "\nY: ", tokenizer.decode(y, skip_special_tokens=False),
    #        "\nAttention_mask: ", attention_mask, 
    #        "\nToken_type_id: ", token_type_id, 
    #        "\nisNext: ", isNext , "\n\n")
    
    # convert Tensor
    x_tensor = torch.tensor(x, dtype=torch.int16)
    y_tensor = torch.tensor(y, dtype=torch.int16)
    att_mask_tensor = torch.tensor(attention_mask, dtype=torch.int16)
    token_type_tensor = torch.tensor(token_type_id, dtype=torch.int16)
    isNext_tensor = torch.tensor(isNext, dtype=torch.int16)

    return {"X": x_tensor, "Y": y_tensor, "Attention_mask": att_mask_tensor, "Token_type_id":token_type_tensor, "isNext": isNext_tensor}


In [33]:
nsp_mlm_table = a_b_table.map(create_nsp_mlm_table).filter(del_none_filter).remove_columns(["A", "B"])

Map:   0%|          | 0/4119470 [00:00<?, ? examples/s]

could not found word
len of tokes:  9
could not found word
len of tokes:  6
could not found word
len of tokes:  9
could not found word
len of tokes:  8
could not found word
len of tokes:  16
could not found word
len of tokes:  10
could not found word
len of tokes:  10
could not found word
len of tokes:  8
could not found word
len of tokes:  11
could not found word
len of tokes:  10
could not found word
len of tokes:  9
could not found word
len of tokes:  7
could not found word
len of tokes:  9
could not found word
len of tokes:  15
could not found word
len of tokes:  8
could not found word
len of tokes:  10
could not found word
len of tokes:  7
could not found word
len of tokes:  10
could not found word
len of tokes:  7
could not found word
len of tokes:  8
could not found word
len of tokes:  9
could not found word
len of tokes:  12
could not found word
len of tokes:  10
could not found word
len of tokes:  9
could not found word
len of tokes:  8
could not found word
len of tokes:  7
co

Filter:   0%|          | 0/4119470 [00:00<?, ? examples/s]

In [34]:
nsp_mlm_table.save_to_disk(trwiki_path_prefix + "nsp_mlm_table")

Saving the dataset (0/17 shards):   0%|          | 0/4119470 [00:00<?, ? examples/s]

In [10]:
x = nsp_mlm_table["X"]

In [14]:
d = {"ss":x}

In [15]:
import pandas as pd
asd = pd.DataFrame(d)

In [17]:
asd.isnull().sum() 

ss    1943
dtype: int64

In [34]:
nsp_mlm_table = nsp_mlm_table.filter(del_none_filter)

Filter:   0%|          | 0/4119470 [00:00<?, ? examples/s]

In [9]:
nsp_mlm_table = load_from_disk(trwiki_path_prefix + "nsp_mlm_table") 

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

In [9]:
nsp_mlm_table[0]

{'A': "Thomas Lüthi (d. 6 Eylül 1986; Oberdiessbach), İnterwetten Paddock Moto2 takımı için, Moto2 Grand Prix Dünya Şampiyonası'nda yarışan İsviçreli profesyonel bir motosiklet yol yarışçısı.",
 'B': "2010 yılından bu yanan Moto2 Dünya Şampiyonası'nda yarışmaktadır.",
 'X': [2,
  5759,
  12618,
  2327,
  12,
  44,
  18,
  26,
  1883,
  21,
  4,
  4,
  26,
  31,
  16769,
  8797,
  10743,
  10543,
  13,
  16,
  1360,
  1253,
  3681,
  4678,
  510,
  14310,
  2943,
  1584,
  2575,
  522,
  22,
  2354,
  1072,
  16,
  2575,
  522,
  22,
  5297,
  7390,
  1490,
  3425,
  11,
  1573,
  12709,
  13776,
  2591,
  962,
  14869,
  1505,
  1694,
  9041,
  18,
  3,
  22,
  20,
  4,
  20,
  2911,
  4,
  22236,
  2575,
  522,
  22,
  1490,
  3425,
  11,
  1573,
  4643,
  11871,
  8859,
  506,
  18,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [93]:
deneme = nsp_mlm_table[:100]
deneme.keys()

dict_keys(['A', 'B', 'X', 'Y', 'Attention_mask', 'Token_type_id', 'isNext'])

In [97]:
x_tensor = torch.tensor(deneme["X"], dtype=torch.int16)
y_tensor = torch.tensor(deneme["Y"], dtype=torch.int16)
att_mask = torch.tensor(deneme["Attention_mask"], dtype=torch.int16)
token_type = torch.tensor(deneme["Token_type_id"], dtype=torch.int16)
isNext = torch.tensor(deneme["isNext"], dtype=torch.int16)

In [46]:
a.dtype

torch.float16

In [47]:
a.shape

torch.Size([100, 255])

In [ ]:
# önce bir dataloader kullanarak hız tespitinde bulun (+ shuffle beklediğin gibi mi olmuş test et: tekrar decode'la batch'i) [fonk'laştır veri görselleştirme mantığı]
# tensor dönüşümü yapılacak, o şekilde kaydedilecek (fonklar vs de ona göre kaydedilecek)
# split yapılabilecek

In [204]:
def print_examples(x, y, att_mask=None, token_type=None, is_next=None):
    if isinstance(x, torch.Tensor):
        x, y = x.tolist(), y.tolist()

    assert len(x) == len(y), "x and y are not equal..."

    for i in range(len(x)):
        print("x: ", tokenizer.decode(x[i], skip_special_tokens=False))
        print("y: ", tokenizer.decode(y[i], skip_special_tokens=False))

        if att_mask is not None:
            print("att_mask: ", att_mask[i])
        if token_type is not None:
            print("token_type: ", token_type[i])
        if is_next is not None:
            print("isNext: ", is_next[i])
        print("\n\n")


In [100]:
# print_examples(x_tensor, y_tensor, att_mask, token_type, isNext)
print_examples(x_tensor, y_tensor, is_next=isNext)

x:  [CLS] thomas lüthi ( d. 6 eylül 1 [MASK] [MASK] 6 ; oberdiessbach ), i̇nterwetten paddock moto 2 takımı için, moto 2 grand prix dünya şampiyonası ' nda yarışan i̇sviçreli profesyonel bir motosiklet yol yarışçısı. [SEP] 2 0 [MASK] 0 yılından [MASK] yanan moto 2 dünya şampiyonası ' nda yarışmaktadır. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [24]:
from dataclasses import dataclass, asdict

In [19]:
@dataclass
class Point:
    vocab_size: int = 25_000
    limit_alpabet: int = 500
    min_frequency: int = 5

In [20]:
d = Point(**deneme)
d

Point(vocab_size=25000, limit_alpabet=500, min_frequency=65)

In [13]:
deneme = {"min_frequency": 65}

d.min_frequency = deneme

In [25]:
a = asdict(d)

In [26]:
a

{'vocab_size': 25000, 'limit_alpabet': 500, 'min_frequency': 65}

## Model Time

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import math
import os


from datasets import load_from_disk
import numpy as np
import re

from tokenizers import Tokenizer 

In [2]:
tokenizer = Tokenizer.from_file("wordpiece_tokenizer.json")

In [3]:
pad_id = tokenizer.token_to_id("[PAD]")
pad_id

1

In [6]:
def count_pattern_occurrences(string, pattern):
    count = 0
    index = string.find(pattern)
    while index != -1:
        count += 1
        index = string.find(pattern, index + len(pattern))
    return count

In [80]:


dd = load_from_disk("data/tr_wiki67/nsp_mlm_table")
a = np.array(dd[0]["Attention_mask"])
b = np.array(dd[0]["Token_type_id"])
stringimiz = tokenizer.decode(dd[0]["X"], skip_special_tokens=False)
pattern = "[PAD]"

count_pattern_occurrences(stringimiz, pattern), (a == 0).sum(), stringimiz, a, b


Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

(201,
 201,
 "[CLS] thomas lüthi ( d. [SEP] 6 eylül mutasyon [MASK] 8 6 ; oberdiessbach ), i̇nterwettei paddock motn 2 takımı için, motn 2 grand prix dünya [MASK] ' nda yarışan i̇sviçreli profesyonel bir motosiklet yol yarışçısı. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [4]:
class PositionalEmbedding(torch.nn.Module):

    def __init__(self, d_model, max_len):
        super(PositionalEmbedding, self).__init__()

        pe = torch.zeros(max_len, d_model).float()
        pe.requires_grad = False

        for pos in range(max_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10_000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        
        # add batch dimension
        self.pe = pe.squeeze(0)
    
    def forward(self, x):
        return self.pe
    
class BERTEmbedding(torch.nn.Module):
    """
    TokenEmbedding, PositionalEmbedding, SegmentEmbedding
    """
    def __init__(self, vocab_size, embed_size, seq_len, dropout):
        super(BERTEmbedding, self).__init__()
        # padding_idx is not updated during training!
        self.token = torch.nn.Embedding(vocab_size, embed_size, padding_idx=pad_id)
        self.segment = torch.nn.Embedding(3, embed_size, padding_idx=pad_id)
        self.position = PositionalEmbedding(d_model=embed_size, max_len=seq_len)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, token_type_ids):
        # x, token_type_ids: (B, T)
        
        x = self.token(x) + self.position(x) + self.segment(token_type_ids)
        return self.dropout(x)
    
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, n_heads, d_model, dropout):
        super(MultiHeadAttention, self).__init__()

        assert d_model % n_heads == 0

        self.d_model = d_model
        self.d_k = d_model // n_heads   # head dimensionality
        self.n_heads = n_heads
        self.dropout = dropout

        # key, query, value projections for all heads
        self.qkv_proj = nn.Linear(d_model, d_model * 3)
        # output projection
        self.out_proj = nn.Linear(d_model, d_model)
        # regularization
        self.drop = nn.Dropout(dropout)

    def forward(self, x, attention_mask):
        B, T, C = x.shape   # batch size, sequence length, embedding dimensionality (model_d)

        q, k, v = self.qkv_proj(x).split(self.d_model, dim=2)
        q = q.view(B, T, self.n_heads, self.d_k).transpose(1, 2)    # (B, nh, T, dk)
        k = k.view(B, T, self.n_heads, self.d_k).transpose(1, 2)    # (B, nh, T, dk)
        v = v.view(B, T, self.n_heads, self.d_k).transpose(1, 2)    # (B, nh, T, dk)
        
        # (B, nh, T, T)
        scores = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.shape[-1]))

        # (B, T) -> (B, 1, 1, T) -> (B, 1, T, T)
        attention_mask = attention_mask.unsqueeze(1).unsqueeze(1).expand(B, 1, T, T)
        
        # mask shape: (B, 1, T, T)
        scores = scores.masked_fill(attention_mask == 0, float("-inf"))

        weights = F.softmax(scores, dim=-1)
        weights = self.drop(weights)

        # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        context = weights @ v
        context = context.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection and return
        return self.drop(self.out_proj(context))
    
class FeedForward(nn.Module):
    def __init__(self, d_model, middle_dim, dropout):
        super(FeedForward, self).__init__()

        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(p=dropout)
        self.actv = nn.GELU()

    def forward(self, x):
        out = self.actv(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out
    
class EncoderBlock(nn.Module):
    def __init__(self, d_model, n_heads, ffn_hidden, dropout):
        super(EncoderBlock, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.multihead = MultiHeadAttention(n_heads, d_model, dropout)
        self.ffn = FeedForward(d_model, middle_dim=ffn_hidden, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attention_mask):
        # x shape: (B, T, C)
        # mask shape: (B, T)
        # x_prev shape: (B, T, C)
        x_prev = self.multihead(x, attention_mask)
        
        # residual layer
        x = self.layernorm(x + x_prev)
        # bottleneck (ffn part)
        ffn_out = self.dropout(self.ffn(x))
        x = self.layernorm(ffn_out + x) # second residual
        return x
    
class BERTBackbone(nn.Module):
    def __init__(self, config):
        super(BERTBackbone, self).__init__()
        self.d_model = config.d_model
        self.n_layers = config.n_layer
        self.n_heads = config.n_heads
        self.block_size = config.block_size
        self.ffn_hidden = config.d_model * 4
        self.dropout = config.dropout
 
        
        self.embedding = BERTEmbedding(vocab_size=config.vocab_size, embed_size=self.d_model,
                                       dropout=self.dropout, seq_len=self.block_size)
        self.encoder_blocks = nn.ModuleList(
            [EncoderBlock(self.d_model, self.n_heads, self.ffn_hidden, self.dropout) for _ in range(self.n_layers)]
        )

    def forward(self, x, attention_mask, token_type_ids):
        # x, att_mask, token_type_ids : (B, T)

        # (B, T) -> (B, T, C)
        x = self.embedding(x, token_type_ids)

        for encoder in self.encoder_blocks:
            x = encoder.forward(x, attention_mask)
        return x
    
class NextSentencePredictionHead(nn.Module):

    def __init__(self, d_model):
        super(NextSentencePredictionHead, self).__init__()
        
        self.linear = nn.Linear(d_model, 2)
        # self.loss_NSP = nn.CrossEntropyLoss()

    def forward(self, x, y=None):
        # x (backbone logits): (B, 1, C) 
        # y: (B, 1)

        loss = None
        logits = self.linear(x)

        # logits: (B, 1, 2) -> (B, 2)
        logits = logits.squeeze(1)

        if y is not None:
            # loss = self.loss_NSP(logits, y)
            loss = F.cross_entropy(logits, y)
            return loss, logits
        
        return logits

        
    
class MaskedLanguageModelHead(nn.Module):
    def __init__(self, d_model, vocab_size):
        super(MaskedLanguageModelHead, self).__init__()

        self.linear = nn.Linear(d_model, vocab_size)
        # self.loss_ML = nn.CrossEntropyLoss()

    def forward(self, x, y=None):
        # x (backbone logits) : (B, T-1, C)
        # y: (B, T-1)

        loss = None
        logits = self.linear(x)
        
        # logits: (B, T-1, vocab_size)

        if y is not None:
            # loss = self.loss_ML(logits, y, ignore_index=pad_id)
            # (B, T-1, vocab_size) -> (N, vocab_size), (B, T) -> (N, 1)  view ops for F.cross_entropy compability

            loss = F.cross_entropy(logits.reshape(-1, logits.shape[-1]), y.reshape(-1), ignore_index=pad_id)
            return loss, logits
        return logits
        
    
class BERT_MLM_NSP(nn.Module):
    """
    wrapper of bert backbone + heads (nsp, mlm)
    """
    def __init__(self, config):
        super(BERT_MLM_NSP, self).__init__()

        self.bert_backbone = BERTBackbone(config)
        self.mlm_head = MaskedLanguageModelHead(config.d_model, config.vocab_size)
        self.nsp_head = NextSentencePredictionHead(config.d_model)

    def forward(self, x, attention_mask, token_type_ids, y=None):
        # x, attention_mask, token_type_ids, y: (B, T)

        # (B, T, C)
        backbone_out = self.bert_backbone(x, attention_mask, token_type_ids)

        if y is not None:
            # nsp logits shape: (B, 1, 2)
            nsp_loss, nsp_logits = self.nsp_head(backbone_out[:, 0, :], y[:, 0])
            # mlm logits shape: (B, T-1, vocab_size)
            mlm_loss, mlm_logits = self.mlm_head(backbone_out[:, 1:, :], y[:, 1:])

            return nsp_loss, mlm_loss, nsp_logits, mlm_logits
        
        nsp_logits = self.nsp_head(backbone_out[:, 0, :])
        mlm_logits = self.mlm_head(backbone_out[:, 1:, :])
        return nsp_logits, mlm_logits 

    def get_num_params(self):
        return sum(p.numel() for p in self.parameters())
    
    
    def save_checkpoint(self, name):
        """
        creates and saves checkpoint
        """
        ckpt_path = "models/" + name + "_checkpoint.pth"

        if os.path.exists(ckpt_path):
            print("checkpoint is already exists, change the name...")
        else:
            torch.save({
            "model_state_dict": self.state_dict()
            }, ckpt_path)
            print("checkpoint is saved...")

        
    @staticmethod
    def load_checkpoint(name):
        """
        returns checkpoint
        """
        ckpt_path = "models/" + name + "_checkpoint.pth"
        ckpt = None

        if os.path.exists(ckpt_path):
            ckpt = torch.load(ckpt_path)
            print("checkpoint is loaded...")
        else:
            print("the checkpoint is not founded...")

        return ckpt
        

In [5]:
from dataclasses import dataclass

class BertModelConfig:
    d_model: int = 768
    n_layer: int = 3 
    n_heads: int = 12
    block_size: int = 255
    vocab_size: int = tokenizer.get_vocab_size()
    dropout = float = 0.1
    

In [6]:
trwiki_path_prefix = "data/tr_wiki67/"
raw_table_path = trwiki_path_prefix + "raw_dataset"
ab_table_path = trwiki_path_prefix + "deneme_ab_tablosu"
nsp_mlm_table_path = trwiki_path_prefix + "nsp_mlm_table"

In [7]:
class Dataset_NSP_MLM(Dataset):
    def __init__(self, nsp_mlm_table_path):
        self.nsp_mlm_table = load_from_disk(nsp_mlm_table_path)

    def __len__(self):
        return len(self.nsp_mlm_table)
    
    def __getitem__(self, index):
        row = self.nsp_mlm_table[index]
        # x = torch.tensor(row["X"], dtype=torch.long)
        # y = torch.tensor(row["Y"], dtype=torch.long)
        # att_mask = torch.tensor(row["Attention_mask"], dtype=torch.long)
        # token_type_id = torch.tensor(row["Token_type_id"], dtype=torch.long)
        # is_next = torch.tensor(row["isNext"], dtype=torch.long)

        x = torch.tensor(row["X"])
        y = torch.tensor(row["Y"])
        att_mask = torch.tensor(row["Attention_mask"])
        token_type_id = torch.tensor(row["Token_type_id"])
        is_next = torch.tensor(row["isNext"])

        return x, y, att_mask, token_type_id, is_next


In [8]:
dataset = Dataset_NSP_MLM(nsp_mlm_table_path)

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [9]:
x, y, mask, type_id, isnext = dataset[55]

In [10]:
x.shape, y.shape, type_id.shape, isnext.shape, x.dtype, y.dtype, type_id.dtype, isnext.dtype

(torch.Size([255]),
 torch.Size([255]),
 torch.Size([255]),
 torch.Size([]),
 torch.int64,
 torch.int64,
 torch.int64,
 torch.int64)

In [12]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

x, y, mask, token_type, isnext = next(iter(dataloader))

In [13]:
def print_examples(x, y, att_mask=None, token_type=None, is_next=None, seq_len=255):
    # TODO: çok çirkin oldu temize çek
    
    batch_len = 0
    x_string = ""
    y_string = ""

    if isinstance(x, torch.Tensor):
        batch_len = x.size(0)
        print(batch_len)
        x, y = x.tolist(), y.tolist()
   

    for i in range(batch_len):
        for j in range(seq_len):
            x_string += " " + tokenizer.id_to_token(x[i][j])
            y_string += " " + tokenizer.id_to_token(y[i][j])

        print(x_string)
        print(y_string)
        x_string = ""
        y_string = ""
        if att_mask is not None:
            print("att_mask: ", att_mask[i])
        if token_type is not None:
            print("token_type: ", token_type[i])
        if is_next is not None:
            print("isNext: ", is_next[i])
        print("\n\n")

In [14]:
print_examples(x[:3], y[:3], is_next=isnext[:3])

3
 [CLS] 1 1 . yüzyılda i . aleksios ' un kayın ##val ##ides ##i maria do ##uka ##ina tarafından yeniden inşa ettir ##ildi . [SEP] ele ##z ##han ( arnavutça : " han ##i i ele ##zi ##t [MASK] ; sırpça : " đ ##ener ##al jan ##ković / [UNK] ј ##ан ##к ##ов ##и ##ћ " ; türkçe : eski " i̇lyas ##han " ) . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [15]:
config = BertModelConfig()
model = BERT_MLM_NSP(config)

In [16]:
mask.shape, x.shape

(torch.Size([64, 255]), torch.Size([64, 255]))

In [17]:
a = torch.Tensor(size=(64, 255))

In [18]:
a.unsqueeze(1).unsqueeze(1).size()

torch.Size([64, 1, 1, 255])

In [19]:
a.unsqueeze(1).unsqueeze(1).expand(64, 1, 255, 255).size()

torch.Size([64, 1, 255, 255])

In [20]:
nsp_logits, mlm_logits = model.forward(x, mask, token_type)

In [21]:
nsp_loss, mlm_loss, nsp_logits, mlm_logits = model.forward(x, mask, token_type, y)

In [35]:
def accuracy(logits, targets):
    return ( torch.sum((torch.argmax(logits, dim=1) == targets[:, 0])) / logits.shape[0] ).item()

In [40]:
torch.randint(0, 10, (5, 1))

tensor([[4],
        [2],
        [9],
        [2],
        [2]])

In [ ]:
def print_examples(x, y, att_mask=None, token_type=None, is_next=None, seq_len=255):
    # TODO: çok çirkin oldu temize çek
    
    batch_len = 0
    x_string = ""
    y_string = ""

    if isinstance(x, torch.Tensor):
        batch_len = x.size(0)
        print(batch_len)
        x, y = x.tolist(), y.tolist()
   

    for i in range(batch_len):
        for j in range(seq_len):
            x_string += " " + tokenizer.id_to_token(x[i][j])
            y_string += " " + tokenizer.id_to_token(y[i][j])

        print(x_string)
        print(y_string)
        x_string = ""
        y_string = ""
        if att_mask is not None:
            print("att_mask: ", att_mask[i])
        if token_type is not None:
            print("token_type: ", token_type[i])
        if is_next is not None:
            print("isNext: ", is_next[i])
        print("\n\n")

In [54]:
def print_pred_and_targets(num_sample, inputs, nsp_logits, mlm_logits, targets):
    """
    prints inputs, predictions and actual targets in textual form

    """
    B, T, _ = nsp_logits.size()

    predictions = []
    expected = []   # targets
    input_print = []

    assert num_sample <= B or num_sample > 0

    idx = torch.randperm(B)[:num_sample]

    nsp_logits, mlm_logits, targets = nsp_logits[idx], mlm_logits[idx], targets[idx]
    
    # (num_sample, T-1, C) -> (num_sample, T-1)
    mlm_predictions = torch.argmax(mlm_logits, dim=-1)
    # (num_sample, 1, 2) -> (num_sample, 1)
    nsp_predictions = torch.argmax(nsp_logits, dim=-1)
    # (num_sample, T)
    prediction_tensor = torch.concat(nsp_predictions, mlm_predictions, dim=1)

    for idx in range(num_sample):
        for t in range(T):
            if t == 0:
                input_print += "inputs: "

                if prediction_tensor[idx, t] == 1:
                    predictions += "prediction: isNext-- "
                else:
                    predictions += "prediction: isNotNext-- "

                if targets[idx, t] == 1:
                    expected += "targets: isNext--"
                else:
                    expected += "targets: isNotNext--"
                    
            
            predicted_token = tokenizer.id_to_token(prediction_tensor[idx, t].item())
            input_token = tokenizer.id_to_token(inputs[idx, t].item())
            target_token = tokenizer.id_to_token(targets[idx, t].item())

            if target_token != "[PAD]":
                input_print += input_token
                predictions += predicted_token
                expected += target_token
    print(input_print)
    print(input_print)
    print(input_print)
    
    return input_print, predictions, expected



    decoded_preds = [for each_seq in range(num_sample) for i in range()]



IndentationError: expected an indented block after 'for' statement on line 20 (3872278910.py, line 21)

In [52]:
mlm_logits.size(), y[:, 1:].size()

(torch.Size([64, 254, 25000]), torch.Size([64, 254]))

In [53]:
print_pred_and_targets(10, mlm_logits, y)

['şaş durum kesıy joss sekizinci eşey მour rahatsızlığınaz simg harab tahrir modernurlar otonom masumzan konsolesis oklahσ fes shi yılın siyasetçidir memphis lâ gerekçesiyleif arena davanın endeksi oklahemeyeceğiniury başındaki sonuçlarına serideba onlarlaetti kelimesinin tetk yetiş tamamıyla kost gerçekleştirilmiştir torresamura ayrılır metropolitan ayrılırıyıyıy hür mekângaj sürümcon sıralardadam memphis memphis memphis ekonom memphis memphis faal memphis memphis kro memphis müfrez memphis memphis memphis ped kayıt kro memphis müfrez ped ped montgomery proteinlerin memphis memphis ped memphis müfrez ped memphis ped ped ped ped veba memphis memphis memphis kayıt ped memphis müfrez memphis memphis kayıt ped konuşulmaktadır memphis kro memphis memphis memphisğinde ped ped mas proteinlerin ped boli memphis havaalan memphis ped eserlerine memphis kro memphis ped memphis memphis memphis memphis ped memphis memphis memphis memphis memphis yaratılan ped memphis şimşek memphis memphis memphis

In [37]:
print(f"accuracy of nsp: {accuracy(logits=nsp_logits, targets=y)}...")
print(f"nsp loss: {nsp_loss}")
print(f"mlm loss: {mlm_loss}")

accuracy of nsp: 0.515625...
nsp loss: 0.8355710506439209
mlm loss: 10.315203666687012
